# EECS 545 (WN 2023) Homework 5: ICA

<span class="instruction">Before starting the assignment, please fill in the following cell.</span>

In [1]:
###################################################################
# Enter your first and last name, e.g. "John Doe"                 #
# for example                                                     #
# __NAME__ = "Anthony Liu"                                        #
# __UNIQID__ = "anthliu"                                          #
###################################################################
# raise NotImplementedError("TODO: Add your implementation here.")
__NAME__ = "Yuzhou Chen"                                      #
__UNIQID__ = "yzc"  
###################################################################
#                        END OF YOUR CODE                         #
###################################################################

print(f"Your name and email: {__NAME__} <{__UNIQID__}@umich.edu>")
assert __NAME__ and __UNIQID__

Your name and email: Yuzhou Chen <yzc@umich.edu>


# ICA

In this notebook, you will test your ICA implementation from `ica.py` to perform **audio separation**. You will load a sound file with several mixed sources and produce new sound tracks with unmixed sources predicted from ICA.

## Setup code
Before getting started, we need to run some boilerplate code to set up our environment. You'll need to rerun this setup code each time you start the notebook. Let's start by checking whether we are using Python 3.10 or higher.

In [2]:
import sys
if sys.version_info[0] < 3:
    raise Exception("You must use Python 3")

if sys.version_info[1] < 11:
    print("Autograder will execute your code based on Python 3.11 environment. Please use Python 3.11 or higher to prevent any issues")
    print("You can create a conda environment with Python 3.11 like 'conda create --name eecs545 python=3.11'")
    raise Exception("Python 3 version is too low: {}".format(sys.version))
else:
    print("You are good to go")

You are good to go


First, run this cell load the [autoreload](https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html) extension. This allows us to edit `.py` source files, and re-import them into the notebook for a seamless editing and debugging experience.

In [3]:
%load_ext autoreload
%autoreload 2

Then, we run some setup code for this notebook: Import some useful packages and increase the default figure size.

In [4]:
# install required libraries

# need scipy for writing wav files
!pip install numpy matplotlib scipy

# import libraries
import math
import numpy as np
import matplotlib.pyplot as plt

# set figure size
plt.rcParams['figure.figsize'] = (5.0, 4.0)
plt.rcParams['font.size'] = 10
%config InlineBackend.figure_format = 'retina'

In [5]:
from IPython.display import display_html, HTML

display_html(HTML('''
<style type="text/css">
  .instruction { background-color: yellow; font-weight:bold; padding: 3px; }
</style>
'''));

In [6]:
import ica
ica.hello()   # should print "Hello from ica.py!"

Hello from ica.py!


## Load Data

Let's load the sound data. This data contains mixed sound signals from multiple microphones.

In [7]:
from IPython.display import Audio, display

Fs = 11025  # sampling rate
def normalize(dat):
    return 0.99 * dat / np.max(np.abs(dat))

mix = np.loadtxt('data/ica_data.dat')
X = normalize(mix)
print(f'Loaded {X.shape[0] / Fs:.2f} seconds of audio with {X.shape[1]} microphones.')

Loaded 4.85 seconds of audio with 5 microphones.


We can play the sound files directly in our IPython notebook! The following code should produce some widgets that can play each unmixed microphone track.

In [8]:
for i in range(X.shape[1]):
    print(f'Microphone {i}:')
    display(Audio(X[:, i], rate=Fs))

Microphone 0:


Microphone 1:


Microphone 2:


Microphone 3:


Microphone 4:


## Learning the filter *W*

<p class="instruction">Implement the `ica.filter_grad()` function. </p>

This function computes the SGD update for ICA. This function will be used with `ica.unmixer` to find a filter `W` that we can use to unmix our sources `X`.

The following code should take around up to 1~2 minutes to run.

In [9]:
%%time

W = ica.unmixer(X)
print('W solution:\n', W)

Separating tracks ...
working on alpha = 0.1
working on alpha = 0.1
working on alpha = 0.1
working on alpha = 0.05
working on alpha = 0.05
working on alpha = 0.05
working on alpha = 0.02
working on alpha = 0.02
working on alpha = 0.01
working on alpha = 0.01
working on alpha = 0.005
working on alpha = 0.005
working on alpha = 0.002
working on alpha = 0.002
working on alpha = 0.001
working on alpha = 0.001
W solution:
 [[ 72.15081922  28.62441682  25.91040458 -17.2322227  -21.191357  ]
 [ 13.45886116  31.94398247  -4.03003982 -24.0095722   11.89906179]
 [ 18.89688784  -7.80435173  28.71469558  18.14356811 -21.17474522]
 [ -6.0119837   -4.15743607  -1.01692289  13.87321073  -5.26252289]
 [ -8.74061186  22.55821897   9.61289023  14.73637074  45.28841827]]
CPU times: user 1min, sys: 22.2 s, total: 1min 22s
Wall time: 16.1 s


## Unmixing with *W*
<p class="instruction">Implement the `ica.unmix` function. </p> 

Let's use our `W` to unmix our original sources `X`!

In [10]:
S = normalize(ica.unmix(X, W))

In [11]:
for i in range(X.shape[1]):
    print(f'Unmixed source {i}:')
    display(Audio(S[:, i], rate=Fs))

Unmixed source 0:


Unmixed source 1:


Unmixed source 2:


Unmixed source 3:


Unmixed source 4:


How does it sound? Some overlap in sources may be present, but the sound sources should be pretty clearly separated. If you still hear some significant overlap, something is probably wrong in your implementation...

## Saving the unmixed sources

In [12]:
from scipy.io.wavfile import write

for i in range(X.shape[1]):
    print(f'Saving unmixed track {i}.')
    write(f'ica_unmixed_track_{i}.wav', Fs, S[:, i])

Saving unmixed track 0.
Saving unmixed track 1.
Saving unmixed track 2.
Saving unmixed track 3.
Saving unmixed track 4.


## Submission

<p class="instruction">Report the *W* matrix you found in your final report. Submit this file along with ica.py to the autograder.</p> 
Make sure your unmixed source sound tracks are playable when you submit. These audio files will be embedded in the notebook and we may check these files when grading. You do <b>not</b> need to submit the saved unmixed `wav` tracks!